# Learning from Demonstrations

<img src="https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/refs/heads/main/assets/logo.jpg?raw=true" style="float: left; width: 15%" />

[CSC_53439_EP-2025](https://moodle.ip-paris.fr/course/view.php?id=10716) Lab session #4

2019-2025 Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremiedecock/polytechnique-csc-53439-ep-2025-students/blob/main/lab4_lfd_and_pbrl.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/polytechnique-csc-53439-ep-2025-students/main?filepath=lab4_lfd_and_pbrl.ipynb)

[![NbViewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/jeremiedecock/polytechnique-csc-53439-ep-2025-students/blob/main/lab4_lfd_and_pbrl.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/raw/main/lab4_lfd_and_pbrl.ipynb)

## Introduction

The purpose of this lab is to introduce some classic algorithms of *Learning from Demonstrations*. We will see how they work, their caveats and benefits.

*Learning from Demonstrations* (LfD) is an approach in reinforcement learning where an agent learns behaviors by observing examples provided by a demonstrator. This field is divided into two main branches: *Imitation Learning* and *Inverse Reinforcement Learning* (IRL).

- **Imitation Learning** involves directly mimicking the demonstrator's actions. The agent learns to replicate the demonstrated behavior without attempting to infer the underlying objectives or rewards driving the actions. It focuses on reproducing successful behaviors in a given task, often through supervised learning.

- **Inverse Reinforcement Learning (IRL)**, on the other hand, seeks to infer the demonstrator's underlying reward function. Instead of copying actions, the agent tries to discover the goals or preferences that motivated the demonstrator’s behavior. Once the reward function is learned, the agent can optimize its own policy to achieve similar outcomes.

Both approaches aim to accelerate learning in complex environments by leveraging expert knowledge, but they differ in how they approach the problem of learning from demonstrations.

In the first part of this lab, we will focus on *Imitation Learning* using the *Behavioral Cloning* algorithm. The second part will shift to *Inverse Reinforcement Learning* through the use of the *GAIL* algorithm.

You can either:
- open, edit and execute the notebook in *Google Colab* following this link: https://colab.research.google.com/github/jeremiedecock/polytechnique-csc-53439-ep-2025-students/blob/main/lab4_lfd_and_pbrl.ipynb ; this is the **recommended** choice as you have nothing to install on your computer
- open, edit and execute the notebook in *MyBinder* (if for any reason the Google Colab solution doesn't work): https://mybinder.org/v2/gh/jeremiedecock/polytechnique-csc-53439-ep-2025-students/main?filepath=lab4_lfd_and_pbrl.ipynb
- download, edit and execute the notebook on your computer if Python3 and JypyterLab are already installed: https://github.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/raw/main/lab4_lfd_and_pbrl.ipynb

If you work with Google Colab or MyBinder, **remember to save or download your work regularly or you may lose it!**

## Lab Submission

Please submit your completed notebook in [Moodle : "Lab 4 - Submission"](https://moodle.ip-paris.fr/mod/assign/view.php?id=185691).

### Submission Guidelines

1. **File Naming:** Rename your notebook as follows: **`firstname_lastname-04.ipynb`** where `firstname` and `lastname` match your email address. *Example: `jesse_read-04.ipynb`*
2. **Clear Output Cells:** To reduce file size (**must be under 500 KB**), clear all output cells before submitting. This includes rendered images, videos, plots, and dataframes...
   - **JupyterLab:**
     - Click **"Kernel" → "Restart Kernel and Clear Outputs of All Cells..."**
     - Then go to **"File" → "Save Notebook As..."**
   - **Google Colab:**
     - Click **"Edit" → "Clear all outputs"**
     - Then go to **"File" → "Download" → "Download.ipynb"**
   - **VSCode:**
     - Click **"Clear All Outputs"**
     - Then **save your file**
3. **Upload Your File:** Only **`.ipynb`** files are accepted.

**Note:** Bonus parts (if any) are optional, as their name suggests.

## Setup the Python environment

This notebook relies on several libraries including `torch`, `gymnasium`, `numpy`, `pandas`, `seaborn`, `imageio`, `pygame`, and `tqdm`.
A complete list of dependencies can be found in the following [requirements_lab4.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/main/requirements_lab4.txt) file.

### If you use Google Colab

If you use Google Colab, execute the next cell to install required libraries.

In [ ]:
import sys, subprocess

def is_colab():
    return "google.colab" in sys.modules

def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())

if is_colab():
    # run_subprocess_command("apt install swig xvfb x11-utils")
    run_subprocess_command("pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/main/requirements_lab4_google_colab.txt")

### If you have downloaded the notebook on your computer and execute it in your own Python environment

To set up the necessary dependencies, run the following commands to establish a [Python virtual environment (venv)](https://docs.python.org/3/library/venv.html) that includes all the essential libraries for this lab.

#### On Posix systems (Linux, MacOSX, WSL, ...)

```bash
python3 -m venv env-lab4
source env-lab4/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/main/requirements_lab4.txt
```

#### On Windows

```bash
python3 -m venv env-lab4
env-lab4\Scripts\activate.bat
python3 -m pip install --upgrade pip
python3 -m pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/main/requirements_lab4.txt
```

### Run CSC-53439-EP notebooks locally in a dedicated Docker container

If you are familiar with Docker (or Podman), an image is available on Docker Hub for this lab:

```bash
docker run -it --rm --user root -p 8888:8888 -e NB_UID=$(id -u) -e NB_GID=$(id -g) -v "${PWD}":/home/jovyan/work jdhp/csc-53439-ep-lab4:latest
```

### Import required packages

In [ ]:
import gymnasium as gym
from IPython.display import Video
import json
import lzma
import numpy as np
from numpy.typing import NDArray
import pandas as pd
from pathlib import Path
import torch
from typing import List, Tuple, Deque, Optional, Callable

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
gym.__version__

In [ ]:
sns.set_context("talk")

In [ ]:
FIGS_DIR = Path("figs/")       # Where to save figures (.gif files)
PLOTS_DIR = Path("figs/")      # Where to save plots (.png or .svg files)
MODELS_DIR = Path("models/")   # Where to save models (.pth files)

In [ ]:
if not FIGS_DIR.exists():
    FIGS_DIR.mkdir()
if not PLOTS_DIR.exists():
    PLOTS_DIR.mkdir()
if not MODELS_DIR.exists():
    MODELS_DIR.mkdir()

## PyTorch setup

PyTorch can run on both CPUs and GPUs. The following cell will determine the device PyTorch will use. If a GPU is available, PyTorch will use it; otherwise, it will use the CPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set the device to CUDA if available, otherwise use CPU


For utilizing a GPU on Google Colab, you also have to activate it following the steps outlined [here](https://colab.research.google.com/notebooks/gpu.ipynb).

In [ ]:
print("Available GPUs:")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"- Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("- No GPU available.")

If you have a very recent GPU and want to use it, you might need to install a specific version of PyTorch compatible with your Cuda version.
For this, you will have to edit the [requirements_lab4.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-53439-ep-2025-students/main/requirements_lab4.txt) file and replace the current version of PyTorch with the one compatible with your Cuda version.
Check the [official PyTorch website](https://pytorch.org/get-started/locally/) for more information.

Note that the GPU is not very useful for CartPole (but useful for MuJoCo) because CartPole is a simple and quick problem to solve, and CUDA spends more time transferring data between the CPU and GPU than processing it directly on the CPU.

You can uncomment the next cell to explicitly instruct PyTorch to train neural networks using the CPU.

In [ ]:
# device = "cpu"

In [ ]:
print(f"PyTorch will train and test neural networks on {device}")